# Text Summary Project

Summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI


**Lợi ích:**
1. Không mất phí API – mã nguồn mở
2. Dữ liệu không rời khỏi thiết bị của bạn

**Hạn chế:**
1. Sức mạnh xử lý kém hơn đáng kể so với Frontier Model

## Hướng dẫn cài đặt Ollama

Chỉ cần truy cập  [ollama.com](https://ollama.com) và cài đặt!

Sau khi hoàn tất, máy chủ Ollama sẽ tự động chạy cục bộ.
Bạn có thể kiểm tra bằng cách truy cập:
http://localhost:11434/

Nếu thấy thông báo "Ollama is running", nghĩa là mọi thứ đã hoạt động bình thường.

Nếu không, hãy mở Terminal (Mac) hoặc Powershell (Windows) và nhập lệnh:

ollama serve

Sau đó thử truy cập lại http://localhost:11434/.

In [27]:
# imports

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
# thư viện để lấy dữ liệu từ trang 

In [28]:
# Constants

MODEL = "llama3.2"

In [33]:
# func để lấy nội dung từ trang web
class Website:
    def __init__(self, url):
        self.url = url
        
        # Khởi tạo trình duyệt
        options = webdriver.ChromeOptions()
        options.add_argument("--headless")  # Chạy không hiển thị giao diện
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        
        driver.get(url)  # Truy cập trang web
        html = driver.page_source  # Lấy HTML sau khi JavaScript chạy
        driver.quit()  # Đóng trình duyệt
        
        soup = BeautifulSoup(html, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [25]:
web = Website("https://anthropic.com")
print(web.title)
print(web.text)

Just a moment...
Enable JavaScript and cookies to continue


## Types of prompts

Bạn có thể đã biết điều này – nhưng nếu chưa, bạn sẽ sớm trở nên quen thuộc!

Các mô hình như GPT-4o được huấn luyện để nhận chỉ dẫn theo một cách cụ thể.

Chúng mong đợi:

1. System Prompt
Lời nhắc hệ thống cung cấp thông tin về nhiệm vụ cần thực hiện và giọng điệu nên sử dụng.

2. User Prompt
Lời nhắc từ người dùng – đây là nội dung cuộc hội thoại mà mô hình sẽ phản hồi.

In [ ]:
# Define our system prompt 

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [ ]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "The contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

## Messages

The API from Ollama expects the same message format as OpenAI:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

In [ ]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

## Ollama

In [ ]:
# 

def summarize(url):
    website = Website(url)
    messages = messages_for(website)
    response = ollama.chat(model=MODEL, messages=messages)
    return response['message']['content']

In [ ]:
summarize("https://anthropic.com")

In [ ]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [ ]:
display_summary("https://anthropic.com")

# Let's try more websites


Lưu ý rằng phương pháp này chỉ hoạt động với các trang web có thể được thu thập dữ liệu bằng cách tiếp cận đơn giản này.

Các trang web được render bằng JavaScript, như ứng dụng React, sẽ không hiển thị. Bạn có thể thử triển khai bằng Selenium giúp khắc phục vấn đề này. Bạn cần tìm hiểu cách cài đặt Selenium (có thể hỏi ChatGPT!).

Các trang web được bảo vệ bằng CloudFront (hoặc tương tự) có thể trả về lỗi 403

In [ ]:
display_summary("https://cnn.com")

NameError: name 'web' is not defined